In [2]:
import warnings 
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Data handling
import pandas as pd
pd.set_option('display.max_columns', None)
import openpyxl
import numpy as np
from zipfile import ZipFile
import zipfile
from datetime import timedelta

# Vizualisation (Matplotlib, Plotly, Seaborn, etc. )
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as colors
from plotly.subplots import make_subplots

# EDA (pandas-profiling, etc. )
import scipy.stats as stats
from statsmodels.tsa.stattools import kpss
from scipy.stats import t, ttest_ind, chi2_contingency
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Feature Processing (Scikit-learn processing, etc. )
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Machine Learning (Scikit-learn Estimators, Catboost, LightGBM, etc. )
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score


# Hyperparameters Fine-tuning (Scikit-learn hp search, cross-validation, etc. )


# Other packages
import os
from joblib import dump

In [3]:
holidays=pd.read_csv(r"data/holidays_events.csv", parse_dates =['date'])
oil=pd.read_csv(r"data/oil.csv", parse_dates =['date'])
stores=pd.read_csv(r"data/stores.csv")
train=pd.read_csv(r"data/train.csv", parse_dates =['date'])
transactions=pd.read_csv(r"data/transactions.csv", parse_dates =['date'])

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


- Our test data is without the target variable, which is ok

In [5]:
# check for duplicates

train.duplicated().any(), 
stores.duplicated().any(), 
oil.duplicated().any(),  
holidays.duplicated().any(),
transactions.duplicated().any()

False

In [6]:
# check for missing values in train data

train.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [7]:
# Merge the Stores data to the Train
train_eda = train.copy()
train_eda = train_eda.merge(oil, on=['date'], how='left')
train_eda = train_eda.merge(stores, on=['store_nbr'], how='left')
train_eda = train_eda.merge(holidays, on=['date'], how='left')

In [8]:
train_eda.head()

,id,date,store_nbr,family,sales,onpromotion,dcoilwtico,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,BABY CARE,0.0,0,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,BEAUTY,0.0,0,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,BEVERAGES,0.0,0,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,BOOKS,0.0,0,NaN,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False


In [9]:
# Fill in missing values introduced by the merging
train_eda[['type_y', 'locale', 'locale_name']] = train_eda[['type_y', 'locale', 'locale_name']].fillna('NoHoliday')
train_eda['transferred'] = train_eda['transferred'].fillna(False)

In [10]:
train_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3054348 entries, 0 to 3054347
Data columns (total 16 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
 6   dcoilwtico   float64       
 7   city         object        
 8   state        object        
 9   type_x       object        
 10  cluster      int64         
 11  type_y       object        
 12  locale       object        
 13  locale_name  object        
 14  description  object        
 15  transferred  bool          
dtypes: bool(1), datetime64[ns](1), float64(2), int64(4), object(8)
memory usage: 352.5+ MB


In [11]:
# Columns to drop
columns_to_drop = [ 'city', 'state', 'locale', 'locale_name', 'transferred', 'description']

# Drop the columns
df_merged = train_eda.drop(columns=columns_to_drop)

In [12]:
# Rearrange columns to make 'sales' the last column
cols = [col for col in df_merged.columns if col != 'sales'] + ['sales']
df_merged = df_merged[cols]

In [13]:
# remane columns with relevant names

df_merged.rename(columns = {"cluster":"store_cluster", "dcoilwtico":"oil_price", "type_y":"events", "type_x":"store_type"}, inplace=True)

In [14]:
df_merged.head()

,id,date,store_nbr,family,onpromotion,oil_price,store_type,store_cluster,events,sales
0,0,2013-01-01,1,AUTOMOTIVE,0,NaN,D,13,Holiday,0.0
1,1,2013-01-01,1,BABY CARE,0,NaN,D,13,Holiday,0.0
2,2,2013-01-01,1,BEAUTY,0,NaN,D,13,Holiday,0.0
3,3,2013-01-01,1,BEVERAGES,0,NaN,D,13,Holiday,0.0
4,4,2013-01-01,1,BOOKS,0,NaN,D,13,Holiday,0.0


In [15]:
df_merged.set_index('date', inplace=True)

In [16]:
df_merged.isna().sum()

id                    0
store_nbr             0
family                0
onpromotion           0
oil_price        955152
store_type            0
store_cluster         0
events                0
sales                 0
dtype: int64

In [17]:
# Apply time-based interpolation
df_merged['oil_price'] = df_merged['oil_price'].interpolate(method='index')

In [18]:
df_merged.isna().sum()

id                  0
store_nbr           0
family              0
onpromotion         0
oil_price        1782
store_type          0
store_cluster       0
events              0
sales               0
dtype: int64

In [19]:
# filling missing values

df_merged["oil_price"].fillna(method="bfill", inplace=True)

In [20]:
df_merged.isna().sum()

id               0
store_nbr        0
family           0
onpromotion      0
oil_price        0
store_type       0
store_cluster    0
events           0
sales            0
dtype: int64

#### Preparing Train Data

In [21]:
# check unique values in events column

df_merged["events"].unique()

array(['Holiday', 'NoHoliday', 'Work Day', 'Additional', 'Event',
       'Transfer', 'Bridge'], dtype=object)

In [22]:
# Rename values in events column

df_merged["events"].replace(to_replace={"Bridge", "Event", "Additional"}, value="Holiday", inplace=True) 
df_merged["events"].replace(to_replace={"Work Day", "Transfer"}, value="NoHoliday", inplace=True)

In [23]:
df_merged['events'] = df_merged['events'].replace('Holiday', 1).replace('NoHoliday', 0)

In [24]:
# check for null values

df_merged["events"].isna().sum()

0

In [25]:
# reclassification of product family to reduce number of categorical variables to encode

df_merged["family"].replace(to_replace={"GROCERY I", "GROCERY II", "EGGS", "PRODUCE", "DAIRY", "BREAD/BAKERY", "DELI", "PREPARED FOODS"}, value="GROCERY", inplace=True)
df_merged["family"].replace(to_replace={"HOME AND KITCHEN I", "HOME AND KITCHEN II", "HOME APPLIANCES", "HARDWARE", "PLAYERS AND ELECTRONICS"}, value="HOME AND KITCHEN", inplace=True)
df_merged["family"].replace(to_replace={"MEATS", "POULTRY", "SEAFOOD"}, value="FROZEN FOODS", inplace=True)
df_merged["family"].replace(to_replace={"HOME CARE", "LAWN AND GARDEN", "CLEANING"}, value="HOME CARE AND GARDEN", inplace=True)
df_merged["family"].replace(to_replace={"BEAUTY", "BABY CARE", "LADIESWEAR", "LINGERIE", "PERSONAL CARE"}, value="BEAUTY AND FASHION", inplace=True)
df_merged["family"].replace(to_replace={"LIQUOR,WINE,BEER", "BEVERAGES"}, value="BEVERAGES AND LIQUOR", inplace=True)
df_merged["family"].replace(to_replace={"MAGAZINES", "BOOKS", "CELEBRATION"}, value="SCHOOL AND OFFICE SUPPLIES", inplace=True)
 

In [26]:
# check unique values

df_merged["family"].unique()

array(['AUTOMOTIVE', 'BEAUTY AND FASHION', 'BEVERAGES AND LIQUOR',
       'SCHOOL AND OFFICE SUPPLIES', 'GROCERY', 'HOME CARE AND GARDEN',
       'FROZEN FOODS', 'HOME AND KITCHEN', 'PET SUPPLIES'], dtype=object)

In [27]:
# Check for missing values

df_merged.isna().sum()

id               0
store_nbr        0
family           0
onpromotion      0
oil_price        0
store_type       0
store_cluster    0
events           0
sales            0
dtype: int64

In [28]:
# check cols and rows

df_merged.shape

(3054348, 9)

In [29]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3054348 entries, 2013-01-01 to 2017-08-15
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   id             int64  
 1   store_nbr      int64  
 2   family         object 
 3   onpromotion    int64  
 4   oil_price      float64
 5   store_type     object 
 6   store_cluster  int64  
 7   events         int64  
 8   sales          float64
dtypes: float64(2), int64(5), object(2)
memory usage: 233.0+ MB


##### Feature Engineering

In [30]:
df_merged.index = pd.to_datetime(df_merged.index)

# Create new features

df_merged["Year"] = df_merged.index.year
df_merged["Month"] = df_merged.index.month
df_merged['Day'] = df_merged.index.day
df_merged['Quarter'] = df_merged.index.quarter
df_merged['Week_of_Year'] = df_merged.index.isocalendar().week
df_merged['Day_of_Week'] = df_merged.index.dayofweek
df_merged["Is_Weekend"] = np.where(df_merged['Day_of_Week'] > 4, 1, 0)

In [31]:
df_merged.tail()

,id,store_nbr,family,onpromotion,oil_price,store_type,store_cluster,events,sales,Year,Month,Day,Quarter,Week_of_Year,Day_of_Week,Is_Weekend
date,,,,,,,,,,,,,,,,
2017-08-15,3000883,9,FROZEN FOODS,0,47.57,B,6,1,438.133,2017,8,15,3,33,1,0
2017-08-15,3000884,9,GROCERY,1,47.57,B,6,1,154.553,2017,8,15,3,33,1,0
2017-08-15,3000885,9,GROCERY,148,47.57,B,6,1,2419.729,2017,8,15,3,33,1,0
2017-08-15,3000886,9,SCHOOL AND OFFICE SUPPLIES,8,47.57,B,6,1,121.000,2017,8,15,3,33,1,0
2017-08-15,3000887,9,FROZEN FOODS,0,47.57,B,6,1,16.000,2017,8,15,3,33,1,0


In [32]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3054348 entries, 2013-01-01 to 2017-08-15
Data columns (total 16 columns):
 #   Column         Dtype  
---  ------         -----  
 0   id             int64  
 1   store_nbr      int64  
 2   family         object 
 3   onpromotion    int64  
 4   oil_price      float64
 5   store_type     object 
 6   store_cluster  int64  
 7   events         int64  
 8   sales          float64
 9   Year           int32  
 10  Month          int32  
 11  Day            int32  
 12  Quarter        int32  
 13  Week_of_Year   UInt32 
 14  Day_of_Week    int32  
 15  Is_Weekend     int32  
dtypes: UInt32(1), float64(2), int32(6), int64(5), object(2)
memory usage: 317.5+ MB


In [33]:
df_merged['Week_of_Year'] = df_merged['Week_of_Year'].astype(int)

In [34]:
df_merged.tail()

,id,store_nbr,family,onpromotion,oil_price,store_type,store_cluster,events,sales,Year,Month,Day,Quarter,Week_of_Year,Day_of_Week,Is_Weekend
date,,,,,,,,,,,,,,,,
2017-08-15,3000883,9,FROZEN FOODS,0,47.57,B,6,1,438.133,2017,8,15,3,33,1,0
2017-08-15,3000884,9,GROCERY,1,47.57,B,6,1,154.553,2017,8,15,3,33,1,0
2017-08-15,3000885,9,GROCERY,148,47.57,B,6,1,2419.729,2017,8,15,3,33,1,0
2017-08-15,3000886,9,SCHOOL AND OFFICE SUPPLIES,8,47.57,B,6,1,121.000,2017,8,15,3,33,1,0
2017-08-15,3000887,9,FROZEN FOODS,0,47.57,B,6,1,16.000,2017,8,15,3,33,1,0


In [35]:
df_merged.drop(columns='id', inplace=True)

In [51]:
df_merged['Day_of_Week'].unique()

array([1, 2, 3, 4, 5, 6, 0])

In [36]:
# save to csv

# df_merged.to_csv('data/merged_train_data.csv', index=False)

In [37]:
csv_filename = 'data/merged_train_data.csv'
zip_filename = 'data/merged_train_data.zip'

# Create a ZIP archive and add the CSV file to it
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_filename)


In [52]:
csv_filename = 'data/train.csv'
zip_filename = 'data/train.zip'

# Create a ZIP archive and add the CSV file to it
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_filename)

##### Encoding & Scaling

In [38]:
# Identify numeric and non-numeric columns
num_cols = df_merged.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df_merged.select_dtypes(exclude=[np.number]).columns.tolist()

# Creating imputer variables
numerical_imputer = SimpleImputer(strategy = "mean")
categorical_imputer = SimpleImputer(strategy = "most_frequent")

In [39]:
# resample numeric columns by mean and categorical columns by mode
resampled = df_merged.resample('D').agg({**{col: 'mean' for col in num_cols}, 
                                         **{col: (lambda x: x.mode()[0] if not x.mode().empty else np.nan) for col in cat_cols}}).reset_index()

In [40]:
# Filling missing values in numerical features of training set
resampled[num_cols] = numerical_imputer.fit_transform(resampled[num_cols])

resampled[cat_cols] = categorical_imputer.fit_transform(resampled[cat_cols])

In [41]:
# Calculate the number of rows in the data

n_rows = resampled.shape[0]

# Calculate the split point
split_point = int(n_rows * 0.80)

# Select the first 85% of the rows as the training data
X_train = resampled.iloc[:split_point]
y_train = X_train['sales']
X_train = X_train.drop('sales', axis=1)

# Select the remaining 15% of the rows as the validation data
X_eval = resampled.iloc[split_point:]
y_eval = X_eval['sales']
X_eval = X_eval.drop('sales', axis=1)

In [42]:
num_cols.remove('sales')  # remove 'sales' from num_cols

X_train_cat = X_train[cat_cols].copy()
X_train_num = X_train[num_cols].copy()


X_eval_cat = X_eval[cat_cols].copy()
X_eval_num = X_eval[num_cols].copy()


In [43]:
# Fitting the Imputer
X_train_cat_imputed = categorical_imputer.fit_transform(X_train_cat)
X_train_num_imputed = numerical_imputer.fit_transform(X_train_num)

X_eval_cat_imputed = categorical_imputer.fit_transform(X_eval_cat)
X_eval_num_imputed = numerical_imputer.fit_transform(X_eval_num)

In [44]:
encoder=OneHotEncoder(handle_unknown='ignore')

# encoding the xtrain categories and converting to a dataframe
X_train_cat_encoded = encoder.fit(X_train_cat_imputed)
X_train_cat_encoded = pd.DataFrame(encoder.transform(X_train_cat_imputed).toarray(),
                                   columns=encoder.get_feature_names_out(cat_cols))

# encoding the xeval categories and converting to a dataframe
X_eval_cat_encoded = encoder.fit(X_eval_cat_imputed)
X_eval_cat_encoded = pd.DataFrame(encoder.transform(X_eval_cat_imputed).toarray(),
                                   columns=encoder.get_feature_names_out(cat_cols))


scaler= StandardScaler()

X_train_num_scaled = scaler.fit_transform(X_train_num_imputed)
X_train_num_sc = pd.DataFrame(X_train_num_scaled, columns = num_cols)

X_eval_num_scaled = scaler.fit_transform(X_eval_num_imputed)
X_eval_num_sc = pd.DataFrame(X_eval_num_scaled, columns = num_cols)

X_train_df = pd.concat([X_train_num_sc,X_train_cat_encoded], axis =1)
X_eval_df = pd.concat([X_eval_num_sc,X_eval_cat_encoded], axis =1)

In [45]:
# create a dictionary of models to fit
models = {
    'Random Forest Regressor': RandomForestRegressor(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
}

# iterate over the models and fit each one to the training data
for name, model in models.items():
    model.fit(X_train_df, y_train)
    
# evaluate each model using cross-validation
rmsle_scores = {}
for name, model in models.items():
    scores = cross_val_score(model, X_train_df, y_train, cv=50, scoring='neg_mean_squared_log_error')
    rmsle_scores[name] = np.sqrt(-scores.mean())
    
# print the RMSLE scores for each model
for name, score in rmsle_scores.items():
    print(f'{name}: {score}')

# choose the model with the lowest RMSLE score
best_model_name = min(rmsle_scores, key=rmsle_scores.get)
best_model = models[best_model_name]
print(f'Best model: {best_model_name}')

Random Forest Regressor: 0.24862067549073694
Decision Tree Regressor: 0.30980578528586544
Gradient Boosting Regressor: 0.2328733313994221
Best model: Gradient Boosting Regressor


In [46]:
# set the destination path to the "toolkit" directory
destination = "toolkit"

# create a dictionary to store the objects and their filenames
models = {"numerical_imputer": numerical_imputer,
          "categorical_imputer": categorical_imputer,
          "encoder": encoder,
          "scaler": scaler,
          "Final_model": best_model}

# loop through the models and save them using joblib.dump()
for name, model in models.items():
    dump(model, os.path.join(destination, f"{name}.joblib"))

In [49]:
!pip freeze > requirements.txt
